# G-Mode filtering and inspection using pycroscopy
### Suhas Somnath and Stephen Jesse
The Center for Nanophase Materials Science and The Institute for Functional Imaging for Materials <br>
Oak Ridge National Laboratory<br>
5/05/2017

## Configure the notebook

In [ ]:
# Make sure needed packages are installed and up-to-date
import sys
!conda install --yes --prefix {sys.prefix} numpy scipy matplotlib scikit-learn Ipython ipywidgets h5py
!{sys.executable} -m pip install -U --no-deps pycroscopy
# Current joblib release has some issues, so install from the github repository.
!{sys.executable} -m pip install -U git+https://github.com/joblib/joblib.git@14784b711f2e777e87c857c5a0e8fbd9d5849936

In [ ]:
# Ensure python 3 compatibility
from __future__ import division, print_function, absolute_import, unicode_literals

# Import necessary libraries:
# General utilities:
from os import path

# Computation:
import numpy as np
import h5py

# Visualization:
import matplotlib.pyplot as plt
from IPython.display import display, HTML

# Finally, pycroscopy itself
import pycroscopy as px

# Make Notebook take up most of page width
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

save_plots = False

In [ ]:
# set up notebook to show plots within the notebook
%matplotlib notebook

## Make the data pycroscopy compatible
Converting the raw data into a pycroscopy compatible hierarchical data format (HDF or .h5) file gives you access to the fast fitting algorithms and powerful analysis functions within pycroscopy

#### H5 files:
* are like smart containers that can store matrices with data, folders to organize these datasets, images, metadata like experimental parameters, links or shortcuts to datasets, etc.
* are readily compatible with high-performance computing facilities
* scale very efficiently from few kilobytes to several terabytes
* can be read and modified using any language including Python, Matlab, C/C++, Java, Fortran, Igor Pro, etc.

#### You can load either of the following:
* Any .mat or .txt parameter file from the original experiment
* A .h5 file generated from the raw data using pycroscopy - skips translation

You can select desired file type by choosing the second option in the pull down menu on the bottom right of the file window

In [ ]:
input_file_path = px.io_utils.uiGetFile(caption='Select translated .h5 file or raw experiment data',
                                        file_filter='Parameters for raw G-Line data (*.txt);; \
                                        Translated file (*.h5)')
folder_path, _ = path.split(input_file_path)

if input_file_path.endswith('.txt'):
    print('Translating raw data to h5. Please wait')
    tran = px.GLineTranslator()
    h5_path = tran.translate(input_file_path)
else:
    h5_path = input_file_path

print('Working on:\n' + h5_path)

## Open the .h5 file and extract some basic parameters

In [ ]:
hdf = px.ioHDF5(h5_path)
h5_main = px.hdf_utils.getDataSet(hdf.file, 'Raw_Data')[-1]
h5_spec_vals = px.hdf_utils.getAuxData(h5_main, auxDataName='Spectroscopic_Values')[0]

##### Inspect the contents of this h5 data file
The file contents are stored in a tree structure, just like files on a conventional computer.
The data is stored as a 2D matrix (position, spectroscopic value) regardless of the dimensionality of the data. Thus, the positions will be arranged as row0-col0, row0-col1.... row0-colN, row1-col0.... and the data for each position is stored as it was chronologically collected  

The main dataset is always accompanied by four ancillary datasets that explain the position and spectroscopic value of any given element in the dataset.

Note that G-mode data is acquired line-by-line rather than pixel-by-pixel. 

In [ ]:
print('Datasets and datagroups within the file:\n------------------------------------')
px.io.hdf_utils.print_tree(hdf.file)
 
print('\nThe main dataset:\n------------------------------------')
print(h5_main)
print('\nThe ancillary datasets:\n------------------------------------')
print(hdf.file['/Measurement_000/Channel_000/Position_Indices'])
print(hdf.file['/Measurement_000/Channel_000/Position_Values'])
print(hdf.file['/Measurement_000/Channel_000/Spectroscopic_Indices'])
print(hdf.file['/Measurement_000/Channel_000/Spectroscopic_Values'])

print('\nMetadata or attributes in a datagroup\n------------------------------------')
for key in hdf.file['/Measurement_000'].attrs:
    print('{} : {}'.format(key, hdf.file['/Measurement_000'].attrs[key]))

## Extract necessary parameters:

In [ ]:
parms_dict = h5_main.parent.parent.attrs

samp_rate = parms_dict['IO_rate_[Hz]']
ex_freq = parms_dict['BE_center_frequency_[Hz]']

pixel_ex_wfm = h5_spec_vals[0, :int(h5_spec_vals.shape[1]/parms_dict['grid_num_cols'])]

pts_per_pix = pixel_ex_wfm.size
pts_per_line = h5_main.shape[1]

## Inspect the raw data:

In [ ]:
row_ind = 40
raw_row = h5_main[row_ind].reshape(-1, pts_per_pix)

fig, axes = px.plot_utils.plot_loops(pixel_ex_wfm, raw_row, x_label='Bias (V)', title='Raw Measurement',
                                     plots_on_side=4, y_label='Deflection (a.u.)',
                                     subtitles='Row: ' + str(row_ind) + ' Col:')

## Visualizing information in Fourier space
Visualizing in the fourier space provides information about the noise floor, frequencies which are noise dominant or signal dominant, etc.

This visualization will guide the design of signal filters to remove the noise

In [ ]:
# Preparing the frequency axis:
w_vec = 1E-3*np.linspace(-0.5*samp_rate, 0.5*samp_rate - samp_rate/pts_per_line, pts_per_line)

row_ind = 40
F_resp = np.fft.fftshift(np.fft.fft(h5_main[row_ind]))
fig, ax = plt.subplots(figsize=(12, 7))
ax.axvline(x=1E-3*ex_freq, color='r', linewidth=2, label='Excitation')
ax.plot(w_vec[int(0.5*len(w_vec)):], np.log10(np.abs(F_resp[int(0.5*len(w_vec)):])), label='Response')
ax.set_xlabel('Frequency (kHz)', fontsize=16)
ax.set_ylabel('Amplitude (a.u.)', fontsize=16)
ax.legend(fontsize=14)
ax.set_xscale('log')
ax.set_xlim(ex_freq*1E-4, samp_rate*0.5E-3)
ax.set_title('Noise Spectrum for row ' + str(row_ind), fontsize=16)
px.plot_utils.set_tick_font_size(ax, 14)
if save_plots:
    fig.savefig(os.path.join(other_figures_folder, 
                             'noise_spectrum_line_' + str(row_ind) +'.png'), 
                format='png', dpi=150);

## Try different FFT filters on the data

Good combinations for frequency filters are:
* Just a HarmonicPassFilter
* LowPassFilter + NoiseBandFilter

It is always a good idea to combine these frequency filters with noise thresholding. Try setting noise tolerance values of 1E-6 to 1E-3/

In [ ]:
hpf = px.processing.fft.HarmonicPassFilter(pts_per_line, samp_rate, ex_freq, 1E+3, 10)
lpf = px.processing.fft.LowPassFilter(pts_per_line, samp_rate, 110E+3)
nbf = px.processing.fft.NoiseBandFilter(pts_per_line, samp_rate, [0], [17E+3])

freq_filts = [hpf]
noise_tolerance = 1E-4

# Test filter on a single line:
row_ind = 40
filt_line, fig_filt, axes_filt = px.processing.gmode_utils.test_filter(h5_main[row_ind], 
                                                                       frequency_filters=freq_filts, 
                                                                       noise_threshold=noise_tolerance, 
                                                                       show_plots=True)
if save_plots:
    fig_filt.savefig(path.join(folder_path, 'FFT_filter_on_line_{}.png'.format(row_ind)), format='png', dpi=300)

filt_row = filt_line.reshape(-1, pixel_ex_wfm.size)
# raw_row = h5_main[row_ind].reshape(-1, pts_per_pix)
fig, axes = px.plot_utils.plot_loops(pixel_ex_wfm, filt_row, x_label='Bias (V)', title='FFT Filtering',
                                     plots_on_side=4, y_label='Deflection (a.u.)',
                                     subtitles='Row: ' + str(row_ind) + ' Col:')
if save_plots:
    fig.savefig(path.join(folder_path, 'FFT_filtered_loops_on_line_{}.png'.format(row_ind)), format='png', dpi=300)

## Apply selected filter to entire dataset

In [ ]:
filter_parms = dict()
if freq_filts is not None:
    for filter in freq_filts:
        filter_parms.update(filter.get_parms())
if noise_tolerance is not None:
    filter_parms['noise_threshold'] = noise_tolerance
h5_filt_grp = px.hdf_utils.check_for_old(h5_main, 'FFT_Filtering', new_parms=filter_parms)

if h5_filt_grp is None:
    sig_filt = px.processing.SignalFilter(h5_main, frequency_filters=freq_filts, noise_threshold=noise_tolerance,
                                          write_filtered=True, write_condensed=False, num_pix=1, verbose=True)
    h5_filt_grp = sig_filt.compute()
else:
    print('Taking previously computed results')

h5_filt = h5_filt_grp['Filtered_Data']

In [ ]:
# Test to make sure the filter gave the same results
filt_row = h5_filt[row_ind].reshape(-1, pixel_ex_wfm.size)
fig, axes = px.plot_utils.plot_loops(pixel_ex_wfm, filt_row, x_label='Bias (V)', title='FFT Filtering',
                                     plots_on_side=4, y_label='Deflection (a.u.)',
                                     subtitles='Row: ' + str(row_ind) + ' Col:')

## Now break up the filtered lines into "pixels"
Also visualize loops from different pixels

In [ ]:
# h5_resh = h5_filt_grp['Filtered_Data-Reshape_000/Reshaped_Data']
h5_resh = px.processing.gmode_utils.reshape_from_lines_to_pixels(h5_filt, pixel_ex_wfm.size, 1)
fig, axes = px.plot_utils.plot_loops(pixel_ex_wfm, h5_resh, x_label='Bias (V)', title='FFT Filtering',
                                     plots_on_side=5, y_label='Deflection (a.u.)')
# fig.savefig(path.join(folder_path, 'FFT_filtered_loops_on_line_{}.png'.format(row_ind)), format='png', dpi=300)

In [ ]:
hdf.close()